# Intoduction to Nebula!

This is a step by step guide to hitting the ground running with some differentiable geometry.

In [1]:
import jax
import jax.numpy as jnp

In [2]:
from nebula.workplane import Workplane

def make_box_with_hole(length: jax.Array, hole_length: jax.Array, height: jax.Array) -> Workplane:
    return (
        Workplane.init("XY")
        .rect(length, length)
        .rect(hole_length,hole_length)
        .extrude(height)
    )
length = jnp.array(10)
hole_length = jnp.array(1)
height = jnp.array(2)

box = make_box_with_hole(length, hole_length, height)
box.plot()

# Test with different parameters after JIT

The next run after JIT is a lot faster

In [3]:
box = make_box_with_hole(length, hole_length, height)

In [4]:
from nebula.workplane import Workplane
from nebula.render.visualization import show

profile = (
    Workplane.init("XY")
    .rect(10, 10)
    .rect(10,10, centered=False)
    .extrude(2)
)
profile.plot()

In [5]:
from nebula.workplane import Workplane
from nebula.render.visualization import show

profile = (
    Workplane.init("XY")
    .rect(2, 2, centered=True)
    .rect(0.1, 0.1, centered=True)

    # clockwise validation test
    # .polyline([(0.5, 0, 0), (0, 0.5, 0), (0, 2, 0), (2, 2, 0), (2, 0, 0)])

    .polyline([(2, 0, 0), (2, 2, 0), (0, 2, 0), (0, 0.5, 0), (0.5, 0, 0)])

    .close()
    .extrude(2)
)
profile.plot()

In [6]:
from nebula.workplane import Workplane
from nebula.render.visualization import show

profile = (
    Workplane.init("XY")
    .lineTo(0, 2.0)
    .bspline([(0.5, 2.5), (1.5, 2.5), (2, 2.0)], includeCurrent=True)
    .lineTo(2, 0)
    .close()
    .extrude(2)
)

profile.plot()

# Auto-differentiate Mesh w.r.t Design Variables

In this example we are getting the jacobian magnitudes w.r.t the design variables height and arc height

In [7]:
from nebula.workplane import Workplane
from nebula.render.visualization import Tesselator

def make_mesh(height: jnp.ndarray, arc_height: jnp.ndarray):
    profile = (
        Workplane.init("XY")
        .lineTo(0, height)
        .bspline([
            (0.5, height+arc_height), 
            (1.5, height+arc_height), 
            (2, height)
        ], includeCurrent=True)
        .lineTo(2, 0)
        .close()
        .extrude(2)
    )
    return Tesselator.get_differentiable_mesh(profile)

height = jnp.array(2.0)
arc_height = jnp.array(0.5)

# Differentiate the Geometry with Jax

In [8]:
mesh_jacobian = jax.jacobian(
    lambda height, arc_height: make_mesh(height, arc_height).vertices,
    (0,1)
)(height, arc_height)
jac_magnitude = jnp.linalg.norm(mesh_jacobian[1],  axis=-1)
mesh = make_mesh(height, arc_height)

In [9]:
from nebula.render.visualization import show

show(mesh, "plot", jac_magnitude, name="CAD Sensitivity w.r.t arc height<br><sup>Nebula by Open Orion</sup>")